In [2]:
# 4/9/23
# KM Altenburger

# steps for downloading repo internally; run w/ "pytorch kernel"
# sudo feature install ttls_fwdproxy
# mgt import --src-type pypi sdist
# mgt import --src-type pypi nvidia-cublas-cu11==11.10.3.66
# mgt import --src-type pypi ogb

import sys
sys.path.insert(0, "/home/kaltenburger/fbsource/ogb")
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import ogb
import pandas as pd
import torch

# import torch_geometric
from ogb.nodeproppred import NodePropPredDataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, precision_recall_curve, roc_auc_score
from sklearn.model_selection import train_test_split


Bad key "axes.color_cycle" on line 214 in
/home/kaltenburger/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


I0412 201613.909 _utils_internal.py:187] NCCL_DEBUG env var is set to None


I0412 201613.910 _utils_internal.py:205] NCCL_DEBUG is forced to WARN from None


### read & set-up dataset


In [3]:
dataset = NodePropPredDataset(name='ogbn-proteins')
g = dataset.graph # get edgelist and graph metadata
y = dataset.labels # outcomes
node_species_df = pd.DataFrame({'node_id':range(len(g['node_species'].flatten())), 'node_species': g['node_species'].flatten()})
node_species_df.head()

,node_id,node_species
0,0,3702
1,1,3702
2,2,3702
3,3,3702
4,4,3702


In [4]:
# create edgelist
edge_list_df = pd.DataFrame(g["edge_index"].T)
edge_list_df.columns = ['src','dst']
edge_list_df.head()

,src,dst
0,1,96401
1,96401,1
2,1,68343
3,68343,1
4,1,125069


In [5]:
# sanity check on inter-species connections
edge_list_df_check = edge_list_df.merge(node_species_df, how = 'left', left_on = 'src', right_on = 'node_id')
edge_list_df_check.drop('node_id', axis = 1, inplace=True)
edge_list_df_check = edge_list_df_check.merge(node_species_df, how = 'left', left_on = 'dst', right_on = 'node_id')
edge_list_df_check.drop('node_id', axis = 1, inplace=True)
#edge_list_df_check.head()
print(np.mean(edge_list_df_check['node_species_x']==edge_list_df_check['node_species_y']))
print(np.mean(edge_list_df_check['node_species_x']!=edge_list_df_check['node_species_y']))

0.8059610954678583


0.1940389045321417


In [6]:
y0 = pd.DataFrame(y)
y0 = y0.iloc[:,0:25] # part I
#y0 = y0.iloc[:,25:50] # part II
#y0 = y0.iloc[:,50:112] # part II
y0['node_id'] = range(len(y0[0])) # change here for each part
    
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

X_train = y0['node_id'].iloc[train_idx]
y_train = y0.iloc[train_idx]

X_valid = y0['node_id'].iloc[valid_idx]
X_test = y0['node_id'].iloc[test_idx]
y_valid = y0.iloc[valid_idx]
y_test = y0.iloc[test_idx]



In [7]:
edge_w = pd.DataFrame(g['edge_feat'])
edge_w = edge_w.add_suffix('_weight')
edge_list_df = pd.concat([edge_list_df, edge_w], axis=1)
#edge_list_df.head()
# w0 = edge_w[j]
# edge_list_df['weight'] = w0
# edge_list_df.head()
# # merge in labels for training nodes on a user's connections
relevant_edges_tmp = edge_list_df.merge(y_train, left_on = 'dst', right_on='node_id', how = 'left')
relevant_edges_tmp.head()

,src,dst,0_weight,1_weight,2_weight,3_weight,4_weight,5_weight,6_weight,7_weight,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,node_id
0,1,96401,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,96401,1,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,68343,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68343.0
3,68343,1,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,125069,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
intervals_v1 = relevant_edges_tmp[['0_weight','1_weight', '2_weight', '3_weight','6_weight','4_weight', '5_weight', '7_weight']].quantile([0.75])

In [9]:
np.array(intervals_v1.iloc[0])

array([0.001, 0.001, 0.001, 0.001, 0.001, 0.091, 0.115, 0.177])

In [10]:
from sklearn.preprocessing import Binarizer
k=0
for j in range(8):
    print(j)
    transformer = Binarizer().fit(relevant_edges_tmp[[str(j)+'_weight']])
    relevant_edges_tmp['bin_'+str(k)] = pd.DataFrame(transformer.transform((relevant_edges_tmp[[str(j)+'_weight']]<=np.array(intervals_v1.iloc[0])[j])+0).flatten())
    print(relevant_edges_tmp.head())
    k=k+1
    relevant_edges_tmp['bin_'+str(k)] = pd.DataFrame(transformer.transform((relevant_edges_tmp[[str(j)+'_weight']]>np.array(intervals_v1.iloc[0])[j])+0).flatten())
    k=k+1

0


     src     dst  0_weight  1_weight  2_weight  ...   22   23   24  node_id  bin_0
0      1   96401     0.501     0.001     0.001  ...  NaN  NaN  NaN      NaN      0
1  96401       1     0.501     0.001     0.001  ...  0.0  0.0  0.0      1.0      0
2      1   68343     0.501     0.001     0.001  ...  0.0  0.0  0.0  68343.0      0
3  68343       1     0.501     0.001     0.001  ...  0.0  0.0  0.0      1.0      0
4      1  125069     0.501     0.001     0.001  ...  NaN  NaN  NaN      NaN      0

[5 rows x 37 columns]


1


     src     dst  0_weight  1_weight  ...  node_id  bin_0  bin_1  bin_2
0      1   96401     0.501     0.001  ...      NaN      0      1      1
1  96401       1     0.501     0.001  ...      1.0      0      1      1
2      1   68343     0.501     0.001  ...  68343.0      0      1      1
3  68343       1     0.501     0.001  ...      1.0      0      1      1
4      1  125069     0.501     0.001  ...      NaN      0      1      1

[5 rows x 39 columns]


2


...

     src     dst  0_weight  1_weight  ...  bin_7  bin_8  bin_9  bin_10
0      1   96401     0.501     0.001  ...      0      1      0       1
1  96401       1     0.501     0.001  ...      0      1      0       1
2      1   68343     0.501     0.001  ...      0      1      0       1
3  68343       1     0.501     0.001  ...      0      1      0       1
4      1  125069     0.501     0.001  ...      0      1      0       1

[5 rows x 47 columns]


6


     src     dst  0_weight  1_weight  ...  bin_9  bin_10  bin_11  bin_12
0      1   96401     0.501     0.001  ...      0       1       0       1
1  96401       1     0.501     0.001  ...      0       1       0       1
2      1   68343     0.501     0.001  ...      0       1       0       1
3  68343       1     0.501     0.001  ...      0       1       0       1
4      1  125069     0.501     0.001  ...      0       1       0       1

[5 rows x 49 columns]


7


     src     dst  0_weight  1_weight  ...  bin_11  bin_12  bin_13  bin_14
0      1   96401     0.501     0.001  ...       0       1       0       1
1  96401       1     0.501     0.001  ...       0       1       0       1
2      1   68343     0.501     0.001  ...       0       1       0       1
3  68343       1     0.501     0.001  ...       0       1       0       1
4      1  125069     0.501     0.001  ...       0       1       0       1

[5 rows x 51 columns]


In [11]:
print(relevant_edges_tmp.columns)

Index([     'src',      'dst', '0_weight', '1_weight', '2_weight', '3_weight',
       '4_weight', '5_weight', '6_weight', '7_weight',          0,          1,
                2,          3,          4,          5,          6,          7,
                8,          9,         10,         11,         12,         13,
               14,         15,         16,         17,         18,         19,
               20,         21,         22,         23,         24,  'node_id',
          'bin_0',    'bin_1',    'bin_2',    'bin_3',    'bin_4',    'bin_5',
          'bin_6',    'bin_7',    'bin_8',    'bin_9',   'bin_10',   'bin_11',
         'bin_12',   'bin_13',   'bin_14',   'bin_15'],
      dtype='object')


In [12]:
print('test')

test


In [13]:
tmp = pd.DataFrame(relevant_edges_tmp[['bin_0',    'bin_1',    'bin_2',    'bin_3',    'bin_4',    'bin_5',
        'bin_6',    'bin_7',    'bin_8',    'bin_9',   'bin_10',   'bin_11',
        'bin_12',   'bin_13', 'bin_14',   'bin_15' ]].drop_duplicates())

In [14]:
tmp.reset_index(inplace=True,drop=True)
tmp.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,bin_6,bin_7,bin_8,bin_9,bin_10,bin_11,bin_12,bin_13,bin_14,bin_15
0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0
1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0
2,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1
3,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0
4,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0


In [15]:
tmp['edge_bin'] = 0
tmp['edge_bin'] = pd.DataFrame(range(np.shape(tmp)[0]))

In [16]:
relevant_edges_tmp['edge_bin']=0
for k in np.array(tmp.edge_bin):
    print(k)
    relevant_edges_tmp['edge_bin'][(relevant_edges_tmp.bin_0==tmp.bin_0[k]) & \
(relevant_edges_tmp.bin_1==tmp.bin_1[k]) & \
(relevant_edges_tmp.bin_2==tmp.bin_2[k]) & \
(relevant_edges_tmp.bin_3==tmp.bin_3[k]) & \
(relevant_edges_tmp.bin_4==tmp.bin_4[k]) & \
(relevant_edges_tmp.bin_5==tmp.bin_5[k]) & \
(relevant_edges_tmp.bin_6==tmp.bin_6[k]) & \
(relevant_edges_tmp.bin_7==tmp.bin_7[k]) & \
(relevant_edges_tmp.bin_8==tmp.bin_8[k]) & \
(relevant_edges_tmp.bin_9==tmp.bin_9[k]) & \
(relevant_edges_tmp.bin_10==tmp.bin_10[k]) & \
(relevant_edges_tmp.bin_11==tmp.bin_11[k]) & \
(relevant_edges_tmp.bin_12==tmp.bin_12[k]) & \
(relevant_edges_tmp.bin_13==tmp.bin_13[k]) & \
(relevant_edges_tmp.bin_14==tmp.bin_14[k]) & \
(relevant_edges_tmp.bin_15==tmp.bin_15[k]) ] = tmp.edge_bin[k]

0


<ipython-input-16-5d312f104b87>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



1


2


3


...

123


124


125


126


127


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler


# set-up loop
#rocauc_v1_ud_w = []
rocauc_v2_ud_w_regularized = []
rocauc_v2_ud_w = []
#rocauc_ud_uw = []
for i in range(0,25): # loop through outcomes part 1
#for i in range(25,50):
#for i in range(50,112):
    print(i)
    relevant_edges = relevant_edges_tmp[["src", "dst", "node_id", i, 'edge_bin']]
    relevant_edges.columns = ["src", "dst",  "node_id", "y", 'edge_bin']
    label1 = relevant_edges.groupby(["src", 'edge_bin'])["y"].sum().reset_index() # same by edgebin
    labeltotal = relevant_edges.groupby(["src", 'edge_bin'])["y"].count().reset_index()
    relevant_edgelist = labeltotal.merge(label1, on=["src", 'edge_bin'])
    ## this is already by edge bin
    relevant_edgelist["label_0"] = relevant_edgelist["y_x"] - relevant_edgelist["y_y"]
    relevant_edgelist.rename(columns={"y_y": "label_1"}, inplace=True)
    relevant_edgelist.drop(["y_x"], axis=1, inplace=True)

    ## v0 - linear combination of single layers
    normalizer = pd.DataFrame(y_train[i].value_counts(normalize=True, ascending=True))

    # step 1 - one hop scores
    step1_one_hop_scores = pd.DataFrame(
        {
            "src": relevant_edgelist.src,
            "edge_bin": relevant_edgelist.edge_bin,
            "frac": np.log(
                (relevant_edgelist["label_1"] + 1) / (relevant_edgelist["label_0"] + 1)
            )
            * list((normalizer.iloc[0]) / (normalizer.iloc[1]))[0],
        }
    )
    two_hop_stg_v1_m = relevant_edges.loc[relevant_edges.edge_bin==0].merge(
            step1_one_hop_scores, left_on=["dst",], right_on=["src",], how="inner" # keep edge_bin and one-hop scores same
        )
    two_hop_stg_v1_m.rename(columns={'frac':'frac'+str(k)}, inplace=True)

    two_hop_stg_v1_m = two_hop_stg_v1_m.groupby(['src_x','edge_bin_x','edge_bin_y'])['frac'+str(k)].sum().reset_index()

    for m in np.unique(relevant_edges.edge_bin):
        #print(m)
        two_hop_stg_v1_m = relevant_edges.loc[relevant_edges.edge_bin==m].merge(
            step1_one_hop_scores, left_on=["dst",], right_on=["src",], how="inner" # keep edge_bin and one-hop scores same
        )
        two_hop_stg_v1_m.rename(columns={'frac':'frac'+str(m)}, inplace=True)
        two_hop_stg_v1_m = two_hop_stg_v1_m.groupby(['src_x','edge_bin_x','edge_bin_y'])['frac'+str(m)].sum().reset_index()
        if m == 0:
            t0 = pd.pivot_table(two_hop_stg_v1_m, values = 'frac'+str(m), columns = 'edge_bin_y', index = 'src_x').reset_index().fillna(0)
            t0 = t0.add_prefix(str(m)+'_')
            t0.rename(columns={str(m)+'_src_x':'src_x'}, inplace=True)
        if m>0:
            t1 = pd.pivot_table(two_hop_stg_v1_m, values = 'frac'+str(m), columns = 'edge_bin_y', index = 'src_x').reset_index().fillna(0)
            t1 = t1.add_prefix(str(m)+'_')
            t1.rename(columns={str(m)+'_src_x':'src_x'}, inplace=True)
            t0 = t0.merge(t1, how = 'outer', on = 'src_x') ## confirm how to do merge
    

        two_hop_v1_valid = t0.merge(
                y_valid[['node_id',i]], left_on="src_x", right_on="node_id", how="right"
            )
    #regularized model
    clf = LogisticRegression(penalty='l2',
            solver='lbfgs',
            C=10e20).fit(
            StandardScaler().fit(two_hop_v1_valid.drop(['src_x','node_id',i], axis=1).fillna(0)).transform(two_hop_v1_valid.drop(['src_x','node_id',i], axis=1).fillna(0)), two_hop_v1_valid[i]
            )

    pred_v0 = y_test[['node_id',i]].merge(t0, right_on="src_x", left_on="node_id", how="left")
    print(
        "roc-auc, v2*: ",
        roc_auc_score(
            pred_v0[i],
            #clf.predict_proba(StandardScaler.fit(pred_v0.drop(['node_id','src_x',i],axis=1).fillna(0))[:,1]
            clf.predict_proba(StandardScaler().fit(pred_v0.drop(['src_x','node_id',i], axis=1).fillna(0)).transform(pred_v0.drop(['src_x','node_id',i], axis=1).fillna(0)))[:,1]
        ),
    )
    rocauc_v2_ud_w_regularized.append(roc_auc_score(
            pred_v0[i],
            #clf.predict_proba(StandardScaler.fit(pred_v0.drop(['node_id','src_x',i],axis=1).fillna(0))[:,1]
            clf.predict_proba(pred_v0.drop(['src_x','node_id',i], axis=1).fillna(0))[:,1]
        ))



0


roc-auc, v2*:  0.6804185506728198


roc-auc, v2*:  0.6757472780184115


1


roc-auc, v2*:  0.6886667361721668


...

roc-auc, v2*:  0.7242150734614222


4


roc-auc, v2*:  0.7493584559644847


roc-auc, v2*:  0.7322910772921741


5


In [ ]:
# ran .py scripts
# with open('rocauc_v2_ud_w_part1_multilayer.npy', 'wb') as f:
# with open('rocauc_v2_ud_w_part2_multilayer.npy', 'wb') as f:
# with open('rocauc_v2_ud_w_part3_multilayer.npy', 'wb') as f:
#    np.save(f, rocauc_v2_ud_w_regularized)